In [44]:
import os
os.getcwd()

'/home/ubuntu/bowon_ko/kobort'

In [1]:
import csv
from tokenizer import make_tokens
from tqdm import tqdm

answers = []
with open("data/whole_data.tsv", "r", encoding="utf-8") as f:
    wf = open("data/whole_data_bowon2.tsv", "w", encoding="utf-8")
    field_names = ["original_text", "morphed_text", "unk_tokens", "not_unk_tokens", "entity"]
    writer = csv.writer(wf, delimiter='\t')
    writer.writerow(field_names)
    
    lines = csv.reader(f, delimiter='\t',)
    for i, line in tqdm(enumerate(lines)):
        if i>0:
            #print(line)
            if i%1000 == 0:
                print(f"{i+1}문장 완료")
            origin_text, entity = line
            morphed_text = make_tokens(origin_text, model_name="mecab")
            tokens, origin_tokens = make_tokens(origin_text, model_name="wp-mecab")
            tokens_text = ' '.join(tokens)
            origin_tokens_text = ' '.join(origin_tokens)
            writer.writerow([origin_text, morphed_text, tokens_text, origin_tokens_text, entity])

1003it [00:58, 16.76it/s]

1001문장 완료


2003it [01:57, 16.71it/s]

2001문장 완료


3002it [02:55, 16.61it/s]

...

22003it [21:17, 17.61it/s]

22001문장 완료


23003it [22:15, 17.92it/s]

23001문장 완료


23800it [22:59, 17.25it/s]


In [2]:
'''
origin_text : 입력 문장
origin_tokens : tokenization 과정을 거쳤지만 <unk>인 경우에도 <unk>으로 변경 안 한 토큰
tokens : tokenization 과정을 거치고 <unk>인 경우를 <unk>으로 변경한 토큰들 (이전)
labels : origin_tokens, tokens에 대한 라벨
'''
from tokenizer import make_tokens
def return_ner_tagged_sentence_plus(origin_text, labels):
    tokens_with_unk, origin_tokens = make_tokens(text=origin_text, model_name="wp-mecab")
    assert len(tokens_with_unk) == len(origin_tokens)
    assert len(tokens_with_unk) == len(labels)
    assert len(origin_tokens) == len(labels)
    ner_tagged_sentence = ''
    ner_tagged_id_list = []
    
    #divide label per character token
    #space를 제외한 모든 글자에 대한 라벨을 붙이기
    #abc : B-LOC → a : B-LOC, b : I-LOC, c : I-LOC
    char_labels = []
    char_tokens = []
    for token, label in zip(origin_tokens, labels):
        if label == 'O':
            for i in range(len(token)):
                #한글자씩 분해시킨 토큰
                char_token = token[i]
                char_label = 'O'
                if char_token != "#":
                    char_labels.append(char_label)
                    char_tokens.append(char_token)
        elif label.startswith("B-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                if char_token != "#":
                    if i == 0:
                        char_label = "B-"+ entity
                    elif i > 0:
                        char_label = "I-"+ entity
                elif char_token == "#":
                    if i < 2:
                        continue
                    elif i == 2:
                        char_label = "B-"+ entity
                    elif i > 2:
                        char_label = "I-"+ entity
                        
                char_labels.append(char_label)
                char_tokens.append(char_token)
        elif label.startswith("I-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                char_label = "I-"+entity
                if char_token != "#":
                    char_tokens.append(char_token)
                    char_labels.append(char_label)
        
    #merge char tokens
    text_len = len(origin_text)
    left = 0
    right = 0
    char_id = 0 #char_tokens, char_labels 를 스캔하는 id
    
    try:
        while right < text_len: #종료조건 
            #space가 아닌 char에 대해
            if origin_text[right] == ' ':
                ner_tagged_sentence += origin_text[right]
                right += 1
            elif origin_text[right] != ' ':
                #항상 origin_text[right] 와 char_tokens[char_id] 가 동일함
                if char_labels[char_id].startswith('I-'): #잘못된 예측 I가 먼저 나온 상황
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1 #개체명으로 인식하지 않고 넘기기
                    right += 1

                elif char_labels[char_id] == 'O':
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1
                    right += 1

                elif char_labels[char_id].startswith('B-'): #여기서는 작업마치고 left, right 저장해야함
                    entity = char_labels[char_id][2:]
                    pred_label = 'I-'+entity
                    left = right
                    right += 1
                    char_id += 1
                    while True:
                        if origin_text[right] == ' ':
                            right += 1
                        elif origin_text[right] != ' ':
                            if char_labels[char_id] == pred_label:
                                if char_id == len(char_labels)-1:
                                    ner_tagged_sentence += (origin_text[left:right+1]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right,entity])
                                    right += 1
                                    break
                                else:
                                    char_id += 1
                                    right += 1
                            elif char_labels[char_id] != pred_label:
                                if origin_text[right-1] == ' ':
                                    ner_tagged_sentence += (origin_text[left:right-1]+'['+entity+']'+' ')
                                    ner_tagged_id_list.append([left,right-2,entity])
                                else:
                                    ner_tagged_sentence += (origin_text[left:right]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right-1,entity])
                                left = right
                                break
    except:
        print('ERROR')
        print(origin_text)
        
    
    # for char_token, char_label in zip(char_tokens, char_labels):
    #     print(f"token : {char_token}, label : {char_label}")
    return ner_tagged_sentence, ner_tagged_id_list

In [12]:
import csv
import os

with open('data/test2.tsv',"r", encoding="utf-8") as f:
    lines2 = csv.reader(f, delimiter='\t')
    
answers = []
with open('data/test.tsv', "r", encoding="utf-8") as f:
    f2= open('data/test2.tsv',"r", encoding="utf-8")
    lines = csv.reader(f, delimiter='\t',)
    lines2 = csv.reader(f2, delimiter='\t')
    for i, item in enumerate(zip(lines, lines2)):
        line, line2 = item
        if i>0:
            origin_text, _, tokens_text, label_str = line
            _,_,tokens_text2, _, _ = line2
            if tokens_text != tokens_text2:
                print(tokens_text)
                print(tokens_text2)
                print("-"*10)
            
            # labels = label_str.split()
            # tagged_sentence, tagged_id_list = return_ner_tagged_sentence_plus(origin_text, labels)
            # answers.append([origin_text,tagged_sentence])


# for i, answer in enumerate(answers):
#     if i<2:
#         print(answer[0], answer[1])

# with open("data/answer2.tsv", "w", encoding="utf-8") as f:
#     field_names = ["origin_text", "ner_tagged_text"]
#     writer = csv.writer(f, delimiter='\t')
#     writer.writerow(field_names)
#     for answer in answers:
#         writer.writerow([answer[0], answer[1]])
# #            {'origin_text':answer[0], 'ner_tagged_text':answer[1]})
        

지 ##카 바이러스 공포 확산 , 콜롬비아 임신 ##부 2000 여 명 감염 <unk> 중남미 여행자 제 권고
지 ##카 바이러스 공포 확산 , 콜롬비아 임신 ##부 2000 여 명 감염 <unk> 중남미 여행 자제 권 고
----------
슈퍼 리그 우승 팀 인 광저우 에버 그란 ##데 는 2013 년 과 2015 년 AC ##L 정상 에 오른 중국 최강 의 팀 이 다 .
슈퍼 리그 우승 팀 인 광저우 에 버그 란 데 는 2013 년 과 2015 년 AC ##L 정상 에 오른 중국 최강 의 팀 이 다 .
----------
산부인과 의사 ##회 <unk> 송 ##민 ##호 사과 접수 <unk> <unk> 여성 비하 랩 논란 , 바비 우승 부담 ?
산부인과 의 사회 <unk> 송 ##민 ##호 사과 접수 <unk> <unk> 여성 비하 랩 논란 , 바비 우승 부담 ?
----------
서태 ##종 금감원 수석 부원 ##장 은 <unk> 많 은 국민 이 쓰 지 않 는 계좌 를 해지 하 고 싶 어도 자신 의 금융 계좌 가 어느 회사 것 인지 조 차 파악 하 기 곤란 한 경우 가 많 다 <unk> 며 <unk> 이런 휴 ##면 계좌 가 대포 통장 에 악용 되 고 <unk> 잠자 는 돈 <unk> 이 계속 나오 면서 국민 이 금융 재산 을 손실 보 고 있 다 <unk> 고 말 했 다 .
서태 ##종 금감원 수석 부원 ##장 은 <unk> 많 은 국민 이 쓰 지 않 는 계좌 를 해지 하 고 싶 어도 자신 의 금융 계좌 가 어느 회사 것 인지 조차 파악 하 기 곤란 한 경우 가 많 다 <unk> 며 <unk> 이런 휴 ##면 계좌 가 대포 통장 에 악용 되 고 <unk> 잠자 는 돈 <unk> 이 계속 나오 면서 국민 이 금융 재산 을 손실 보 고 있 다 <unk> 고 말 했 다 .
----------
또 <unk> 일부 한국 남성 이 베트남 이나 캄보디아 , 몽골 , 필리핀 에서 아동 성매매 관광 에 관여 하 는 경우 가 있 다 <unk> 고 밝혔 다 .
또 <un

In [7]:
import csv
import os
answers = []
with open('data/test.tsv', "r", encoding="utf-8") as f:
    lines = csv.reader(f, delimiter='\t',)
    for i, line in enumerate(lines):
        if i>0:
            origin_text, _, a, b = line
            a = a.split()
            b = b.split()
            #c = c.split()
            assert len(a) == len(b)

In [20]:
position = ["중구","남구","동구","북구","울주군","학성동", "학산동","복산동","북정동","옥교동","성남동","교동","우정동","성안동","유곡동","태화동","다운동","동동","서동","남외동","장현동","약사동","반구동","반구1동","반구2동","복산1동","복산2동","중앙동","병영1동","병영2동",
           "무거동","옥동","두왕동","신정동","달동","삼산동","여천동","야음동","선암동","상개동","부곡동","고사동","성암동","황성동","용연동","남화동","용잠동","장생포동","매암동","신정1동","신정2동","신정3동","신정4동","신정5동","삼호동","대현동","수암동","야음장생포동",
           ]


In [21]:
tmp = "방어동,화정동,일산동,전하동,미포동,주전동,동부동,서부동,대송동,전하1동,전하2동,남목1동,남목2동,남목3동"
tmp =tmp.split(",")
position = position +tmp
print(position)

['중구', '남구', '동구', '북구', '울주군', '학성동', '학산동', '복산동', '북정동', '옥교동', '성남동', '교동', '우정동', '성안동', '유곡동', '태화동', '다운동', '동동', '서동', '남외동', '장현동', '약사동', '반구동', '반구1동', '반구2동', '복산1동', '복산2동', '중앙동', '병영1동', '병영2동', '무거동', '옥동', '두왕동', '신정동', '달동', '삼산동', '여천동', '야음동', '선암동', '상개동', '부곡동', '고사동', '성암동', '황성동', '용연동', '남화동', '용잠동', '장생포동', '매암동', '신정1동', '신정2동', '신정3동', '신정4동', '신정5동', '삼호동', '대현동', '수암동', '야음장생포동', '방어동', '화정동', '일산동', '전하동', '미포동', '주전동', '동부동', '서부동', '대송동', '전하1동', '전하2동', '남목1동', '남목2동', '남목3동']


In [31]:
site = ['울산광역시','울산','중구', '남구', '동구', '북구', '울주군', '학성동', '학산동', '복산동', '북정동', '옥교동', '성남동', '교동', '우정동', '성안동', '유곡동', '태화동', '다운동', '동동', '서동', '남외동', '장현동', '약사동', '반구동', '반구1동', '반구2동', '복산1동', '복산2동', '중앙동', '병영1동', '병영2동', '무거동', '옥동', '두왕동', '신정동', '달동', '삼산동', '여천동', '야음동', '선암동', '상개동', '부곡동', '고사동', '성암동', '황성동', '용연동', '남화동', '용잠동', '장생포동', '매암동', '신정1동', '신정2동', '신정3동', '신정4동', '신정5동', '삼호동', '대현동', '수암동', '야음장생포동', '방어동', '화정동', '일산동', '전하동', '미포동', '주전동', '동부동', '서부동', '대송동', '전하1동', '전하2동', '남목1동', '남목2동', '남목3동']


In [32]:
tmp = "창평동,호계동,매곡동,가대동,신천동,중산동,상안동,천곡동,달천동,시례동,무룡동,구유동,정자동,신명동,대안동,당사동,신현동,산하동,어물동,명촌동,진장동,연암동,효문동,양정동,화봉동,송정동,염포동,농소1동,농소2동,농소3동,강동동"
site = site + tmp.split(",")
print(site)

['울산광역시', '울산', '중구', '남구', '동구', '북구', '울주군', '학성동', '학산동', '복산동', '북정동', '옥교동', '성남동', '교동', '우정동', '성안동', '유곡동', '태화동', '다운동', '동동', '서동', '남외동', '장현동', '약사동', '반구동', '반구1동', '반구2동', '복산1동', '복산2동', '중앙동', '병영1동', '병영2동', '무거동', '옥동', '두왕동', '신정동', '달동', '삼산동', '여천동', '야음동', '선암동', '상개동', '부곡동', '고사동', '성암동', '황성동', '용연동', '남화동', '용잠동', '장생포동', '매암동', '신정1동', '신정2동', '신정3동', '신정4동', '신정5동', '삼호동', '대현동', '수암동', '야음장생포동', '방어동', '화정동', '일산동', '전하동', '미포동', '주전동', '동부동', '서부동', '대송동', '전하1동', '전하2동', '남목1동', '남목2동', '남목3동', '창평동', '호계동', '매곡동', '가대동', '신천동', '중산동', '상안동', '천곡동', '달천동', '시례동', '무룡동', '구유동', '정자동', '신명동', '대안동', '당사동', '신현동', '산하동', '어물동', '명촌동', '진장동', '연암동', '효문동', '양정동', '화봉동', '송정동', '염포동', '농소1동', '농소2동', '농소3동', '강동동']


In [33]:
tmp = "온산읍,언양읍,온양읍,범서읍,청량읍,삼남읍,서생면,웅촌면,두동면,두서면,상북면,삼동면"
site = site + tmp.split(",")
print(site)

['울산광역시', '울산', '중구', '남구', '동구', '북구', '울주군', '학성동', '학산동', '복산동', '북정동', '옥교동', '성남동', '교동', '우정동', '성안동', '유곡동', '태화동', '다운동', '동동', '서동', '남외동', '장현동', '약사동', '반구동', '반구1동', '반구2동', '복산1동', '복산2동', '중앙동', '병영1동', '병영2동', '무거동', '옥동', '두왕동', '신정동', '달동', '삼산동', '여천동', '야음동', '선암동', '상개동', '부곡동', '고사동', '성암동', '황성동', '용연동', '남화동', '용잠동', '장생포동', '매암동', '신정1동', '신정2동', '신정3동', '신정4동', '신정5동', '삼호동', '대현동', '수암동', '야음장생포동', '방어동', '화정동', '일산동', '전하동', '미포동', '주전동', '동부동', '서부동', '대송동', '전하1동', '전하2동', '남목1동', '남목2동', '남목3동', '창평동', '호계동', '매곡동', '가대동', '신천동', '중산동', '상안동', '천곡동', '달천동', '시례동', '무룡동', '구유동', '정자동', '신명동', '대안동', '당사동', '신현동', '산하동', '어물동', '명촌동', '진장동', '연암동', '효문동', '양정동', '화봉동', '송정동', '염포동', '농소1동', '농소2동', '농소3동', '강동동', '온산읍', '언양읍', '온양읍', '범서읍', '청량읍', '삼남읍', '서생면', '웅촌면', '두동면', '두서면', '상북면', '삼동면']


In [ ]:
tmp = "온산읍,언양읍,온양읍,범서읍,청량읍,삼남읍,서생면,웅촌면,두동면,두서면,상북면,삼동면"
site = site + tmp.split(",")
print(site)

In [39]:
from nltk.tokenize import sent_tokenize

#두 문장 이상으로 구성된 문단 분리
def separate_paragraph(paragraph, max_seq_len = 512):
    sentences = paragraph#sent_tokenize(paragraph)
    result = []
    for sentence in sentences:
        sen = sentence.strip()
        while len(sen) > max_seq_len:
            last_space_id = max_seq_len-1
            while last_space_id > 0:
                if sen[last_space_id] == " ":
                    result.append(sen[:last_space_id])
                    sen = sen[last_space_id:]
                    break
                else:
                    last_space_id -= 1
        result.append(sen)
    return result

In [40]:
print("a"*5)

aaaaa


In [42]:
result = separate_paragraph(['이후 현대엔지니어링은 힐스테이트 브랜드를 통해 주택사업 강자로 떠올랐다. 실제로 현대엔지니어링은 힐스테이트 브랜드를 공유한 이후 분양에서 흥행을 거두기 시작했다. 종합건설업자 시공능력평가 순위는 지난 2014년 10위에서 올해 6위로 도약했다.이같은 포트폴리오 변화는 수치에서도 나타난다. 올해 3분기 사업부문 매출 비중은 플랜트·인프라 부문 42.22%, 건축·주택 부문 45.70%, 기타 부문 12.08%으로 구성된다. 2019년 플랜트·인프라 부문 49.61%, 건축·주택 부문 38.34%와 비교하면 플랜트·인프라 비중이 줄고 건축·주택이 그 자리를 차지한 것이다.'], max_seq_len=20)
print(result)

['이후 현대엔지니어링은 힐스테이트', ' 브랜드를 통해 주택사업 강자로', ' 떠올랐다. 실제로 현대엔지니어링은', ' 힐스테이트 브랜드를 공유한 이후', ' 분양에서 흥행을 거두기 시작했다.', ' 종합건설업자 시공능력평가 순위는', ' 지난 2014년 10위에서 올해', ' 6위로 도약했다.이같은 포트폴리오', ' 변화는 수치에서도 나타난다. 올해', ' 3분기 사업부문 매출 비중은', ' 플랜트·인프라 부문 42.22%,', ' 건축·주택 부문 45.70%,', ' 기타 부문 12.08%으로', ' 구성된다. 2019년', ' 플랜트·인프라 부문 49.61%,', ' 건축·주택 부문 38.34%와', ' 비교하면 플랜트·인프라 비중이', ' 줄고 건축·주택이 그 자리를', ' 차지한 것이다.']
